# Atualizar Tabela ACCESS

In [ ]:
import pandas as pd
import sqlite3
from email import message_from_file
from io import StringIO
import pyodbc  
import os

#Caminho Access

caminho_arquivo_access = r"G"

# Conectar ao banco de dados Access
conn_access = pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + caminho_arquivo_access)
cursor_access = conn_access.cursor()

# Função para carregar as tabelas do Access
def carregar_tabela_access(nome_tabela):
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn_access)
    return df

# Carregar as tabelas do Access
df_tabelas_fracionado = carregar_tabela_access('TABELAS_FRACIONADO')
df_tabelas_lotacao = carregar_tabela_access('TABELAS_LOTACAO')

# Fechar a conexão com o Access
conn_access.close()


# Função para remover colunas indesejadas
def limpar_colunas(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


# Conectar ao banco SQLite
conn = sqlite3.connect('dados_banco.db')
cursor = conn.cursor()

def salvar_tabela(df, nome_tabela):
    # Salva o DataFrame como uma tabela no banco de dados SQLite
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    print(f'Tabela "{nome_tabela}" salva no banco de dados.')   

# Salvar as tabelas do Access
salvar_tabela(df_tabelas_fracionado, 'TABELAS_FRACIONADO')
salvar_tabela(df_tabelas_lotacao, 'TABELAS_LOTACAO')


# Fechar a conexão com o banco de dados após salvar
conn.close()     

# Atualizar dados (Book, Bases apoio)

In [ ]:
import pandas as pd
import sqlite3
from email import message_from_file
from io import StringIO
import pyodbc  
import os


# Caminhos

caminho_arquivo_excel = r"G:"
caminho_arquivo_excel2 = r"G:"


# 1. Carregar as abas do arquivo Excel (utilizando o engine='pyxlsb')
abas = ['Info_Gerais', 'Transp', 'Base_Clientes', 'Base_Produtos']

# Ler os DataFrames para cada aba usando o engine 'pyxlsb'
df_info_gerais = pd.read_excel(caminho_arquivo_excel, sheet_name='Info_Gerais', engine='pyxlsb')
df_transp = pd.read_excel(caminho_arquivo_excel, sheet_name='Transp', engine='pyxlsb')
df_base_clientes = pd.read_excel(caminho_arquivo_excel, sheet_name='Base_Clientes', engine='pyxlsb')
df_base_produtos = pd.read_excel(caminho_arquivo_excel, sheet_name='Base_Produtos', engine='pyxlsb')

# 2. Carregar a aba do segundo arquivo Excel
df_book = pd.read_excel(caminho_arquivo_excel2, sheet_name='Book', engine='pyxlsb')


# Função para remover colunas indesejadas
def limpar_colunas(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


# Conectar ao banco SQLite
conn = sqlite3.connect('dados_banco.db')
cursor = conn.cursor()

def salvar_tabela(df, nome_tabela):
    # Salva o DataFrame como uma tabela no banco de dados SQLite
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    print(f'Tabela "{nome_tabela}" salva no banco de dados.')

# Salvar as tabelas enquanto a conexão estiver aberta
salvar_tabela(df_info_gerais, 'Info_Gerais')
salvar_tabela(df_transp, 'Transp')
salvar_tabela(df_base_clientes, 'Base_Clientes')
salvar_tabela(df_base_produtos, 'Base_Produtos')
salvar_tabela(df_book, 'Book')
# salvar_tabela(df_txt, 'Relatorio_Txt')

# Fechar a conexão com o banco de dados após salvar
conn.close()

Tabela "Info_Gerais" salva no banco de dados.
Tabela "Transp" salva no banco de dados.
Tabela "Base_Clientes" salva no banco de dados.
Tabela "Base_Produtos" salva no banco de dados.
Tabela "Book" salva no banco de dados.


# BI Paletização

In [ ]:
import pandas as pd
import sqlite3
import os

caminho_pasta = r"G:"
caminho_banco = r"G:"
def carregar_txts_pasta(caminho_pasta, separador='|', encoding='utf-8'):
    arquivos_txt = [f for f in os.listdir(caminho_pasta) if f.endswith('.txt')]
    lista_dfs = []
    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            df = pd.read_csv(caminho_arquivo, sep=separador, encoding=encoding)
            lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    if lista_dfs:
        return pd.concat(lista_dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

df_txt_consolidado = carregar_txts_pasta(caminho_pasta)

if not df_txt_consolidado.empty:
    try:
        conn = sqlite3.connect(caminho_banco)
        df_txt_consolidado.to_sql("BI_Paletizacao", conn, if_exists='append', index=False)

        print('✅ Dados dos arquivos .txt inseridos na tabela "BI_Paletizacao".')
        conn.close()
    except Exception as e:
        print(f"❌ Erro ao salvar no banco de dados: {e}")
else:
    print("⚠️ Nenhum dado foi salvo no banco, pois o DataFrame está vazio.")


✅ Dados dos arquivos .txt inseridos na tabela "BI_Paletizacao".


## BI Dedicado

In [ ]:
import pandas as pd
import sqlite3
import os

caminho_pasta = r"G:"
caminho_banco = r"G:"

def carregar_txts_pasta(caminho_pasta, separador='|', encoding='utf-8'):
    arquivos_txt = [f for f in os.listdir(caminho_pasta) if f.endswith('.txt')]
    lista_dfs = []
    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            df = pd.read_csv(caminho_arquivo, sep=separador, encoding=encoding)
            lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    if lista_dfs:
        return pd.concat(lista_dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

df_txt_consolidado = carregar_txts_pasta(caminho_pasta)

if not df_txt_consolidado.empty:
    try:
        conn = sqlite3.connect(caminho_banco)
        df_txt_consolidado.to_sql("BI_Dedicado", conn, if_exists='append', index=False)

        print('✅ Dados dos arquivos .txt inseridos na tabela "BI_Dedicado".')
        conn.close()
    except Exception as e:
        print(f"❌ Erro ao salvar no banco de dados: {e}")
else:
    print("⚠️ Nenhum dado foi salvo no banco, pois o DataFrame está vazio.")


✅ Dados dos arquivos .txt inseridos na tabela "BI_Dedicado".


## BI Estadia

In [ ]:
import pandas as pd
import sqlite3
import os

caminho_pasta = r"G:"
caminho_banco = r"G:"

def carregar_txts_pasta(caminho_pasta, separador='|', encoding='utf-8'):
    arquivos_txt = [f for f in os.listdir(caminho_pasta) if f.endswith('.txt')]
    lista_dfs = []
    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            df = pd.read_csv(caminho_arquivo, sep=separador, encoding=encoding)
            lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    if lista_dfs:
        return pd.concat(lista_dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

df_txt_consolidado = carregar_txts_pasta(caminho_pasta)

if not df_txt_consolidado.empty:
    try:
        conn = sqlite3.connect(caminho_banco)
        df_txt_consolidado.to_sql("BI_Estadia", conn, if_exists='append', index=False)

        print('✅ Dados dos arquivos .txt inseridos na tabela "BI_Estadia".')
        conn.close()
    except Exception as e:
        print(f"❌ Erro ao salvar no banco de dados: {e}")
else:
    print("⚠️ Nenhum dado foi salvo no banco, pois o DataFrame está vazio.")


✅ Dados dos arquivos .txt inseridos na tabela "BI_Estadia".


# BI Descarga

In [ ]:
import pandas as pd
import sqlite3
import os

# Caminhos
caminho_pasta = r"G:"
caminho_banco = r"G:"

# Lista de colunas na ordem correta
colunas_ordenadas = [
    "Data de Requisição", "Nota fiscal", "Série", "Cliente Destinatário", "Cidade_1", "U.F",
    "Tipo de serviço:", "Cobrança por:", "Quantidade:", "Valor unitário:", "N° Requisição", "REF",
    "Título E-mail", "Código", "Destinatário", "Cidade", "UF", "Transportadora", "Tipo de expedição",
    "Data de entrega", "Local de Expedição", "Nfe", "PESO (KG)", "Chave", "Agrupador", "Valor Solicitado",
    "Valor Cobrado por descarga", "Valor Descarga Validado", "Valor Validado por NF", "Procede?", "Franquia?"
]

# Função para carregar os arquivos .txt
def carregar_txts_pasta(caminho_pasta, separador='|', encoding='utf-8'):
    arquivos_txt = [f for f in os.listdir(caminho_pasta) if f.endswith('.txt')]
    lista_dfs = []
    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            df = pd.read_csv(
                caminho_arquivo,
                sep=separador,
                encoding=encoding,
                dtype={"Franquia?": str}
            )
            lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    if lista_dfs:
        df_consolidado = pd.concat(lista_dfs, ignore_index=True)
        # Ordenar colunas
        df_consolidado = df_consolidado[colunas_ordenadas]
        return df_consolidado
    else:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

# Carregar e processar
df_txt_consolidado = carregar_txts_pasta(caminho_pasta)

# Inserir no banco
if not df_txt_consolidado.empty:
    try:
        conn = sqlite3.connect(caminho_banco)
        df_txt_consolidado.to_sql("BI_Descarga", conn, if_exists='append', index=False)
        print('✅ Dados dos arquivos .txt inseridos na tabela "BI_Descarga".')
        conn.close()
    except Exception as e:
        print(f"❌ Erro ao salvar no banco de dados: {e}")
else:
    print("⚠️ Nenhum dado foi salvo no banco, pois o DataFrame está vazio.")


# BI Armazenagem

In [ ]:
import pandas as pd
import sqlite3
import os

caminho_pasta = r"G:"
caminho_banco = r"G:"

def carregar_txts_pasta(caminho_pasta, separador='|', encoding='utf-8'):
    arquivos_txt = [f for f in os.listdir(caminho_pasta) if f.endswith('.txt')]
    lista_dfs = []
    for arquivo in arquivos_txt:
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        try:
            df = pd.read_csv(caminho_arquivo, sep=separador, encoding=encoding)
            lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    if lista_dfs:
        return pd.concat(lista_dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido encontrado na pasta.")
        return pd.DataFrame()

df_txt_consolidado = carregar_txts_pasta(caminho_pasta)

if not df_txt_consolidado.empty:
    try:
        conn = sqlite3.connect(caminho_banco)
        df_txt_consolidado.to_sql("BI_Armazenagem", conn, if_exists='append', index=False)

        print('✅ Dados dos arquivos .txt inseridos na tabela "BI_Armazenagem".')
        conn.close()
    except Exception as e:
        print(f"❌ Erro ao salvar no banco de dados: {e}")
else:
    print("⚠️ Nenhum dado foi salvo no banco, pois o DataFrame está vazio.")


✅ Dados dos arquivos .txt inseridos na tabela "BI_Armazenagem".


## Tabela Valor veiculo

In [ ]:
import pandas as pd
import sqlite3
from email import message_from_file
from io import StringIO
import pyodbc  
import os

# Caminhos
caminho_arquivo_excel = r"G:"

# Lista todas as abas do arquivo
abas_disponiveis = pd.ExcelFile(caminho_arquivo_excel).sheet_names

df_info_veiculo = pd.read_excel(caminho_arquivo_excel, sheet_name='valor')


def limpar_colunas(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

conn = sqlite3.connect('dados_banco.db')
cursor = conn.cursor()

def salvar_tabela(df, nome_tabela):
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    print(f'Tabela "{nome_tabela}" salva no banco de dados.')

salvar_tabela(df_info_veiculo, 'valor_dedicado')

conn.close()

Tabela "valor_dedicado" salva no banco de dados.
